1. RunnableSequence  
作用：构造一个串行的执行链，通过runnable_sequence的实例，调用invoke方法，就等于链当中每一个组件去调用invoke，然后将调用结果传递给下一个组件

In [14]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableSequence

prompt_template = PromptTemplate(input_variables=["user_info"], template="{user_info}")
openai = OpenAI(model="gpt-4o-mini")
output_parser = StrOutputParser()

# runnable_sequence = RunnableSequence(*[prompt_template, openai, output_parser])
runnable_sequence = RunnableSequence(prompt_template, openai, output_parser)



In [21]:
runnable_sequence.invoke("你好，你是谁？")

RateLimitError: Error code: 429 - {'error': {'message': '当前分组上游负载已饱和，请稍后再试 (request id: 20260120184435907228217b8Z2ui5E)', 'type': 'rix_api_error', 'param': '', 'code': None}}

2. RunnableParallel  
具体应用：对于用户输入的同一个问题，我们想要调用不同的大模型进行回答，用户可以比对不同大模型的回答效果

In [19]:
from langchain_core.runnables import RunnableParallel
def func1(a1):
    return a1+"__func1_output"
def func2(a2):
    return a2+"__func2_output"

runnable_parallel = RunnableParallel({"key1": func1, "key2": func2})

In [20]:
runnable_parallel.invoke("你好！")

{'key1': '你好！__func1_output', 'key2': '你好！__func2_output'}

一次请求多个智能体

In [ ]:
from langchain_deepseek import ChatDeepSeek
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
import os

# gpt = OpenAI(model="gpt-4o-mini")
ds = ChatDeepSeek(model="deepseek-chat")
qwen = init_chat_model(
    model="qwen-flash",
    model_provider="openai",
    base_url=os.getenv("DASHSCOPE_BASE_URL"),
    api_key=os.getenv("DASHSCOPE_API_KEY"),
)

messages = [
    ("system", "你是我的英语老师"),
    ("user", "{user_question}")
]
message_template = ChatPromptTemplate.from_messages(messages)

runnable_parallel = message_template | RunnableParallel({"deepseek": ds, "qwen": qwen})

In [ ]:
runnable_parallel.invoke({"user_question": "purpose 和 propose有什么区别？"})

{'deepseek': AIMessage(content='你好！我是你的英语老师，很高兴见到你。  \n无论你是想提高英语水平、准备考试，还是单纯对英语学习感兴趣，我都可以为你提供帮助。  \n\n请告诉我：  \n1. 你目前的英语水平如何？（例如：初级、中级、高级）  \n2. 你学习英语的主要目标是什么？（例如：日常交流、考试、工作等）  \n3. 你希望我从哪些方面帮助你？  \n\n我们可以一起制定学习计划，解决具体问题，或者进行对话练习。期待你的回复！', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 109, 'prompt_tokens': 11, 'total_tokens': 120, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 11}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': '63b6eda4-f06a-493c-916c-798500735f4c', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019bdb19-d8e7-7413-ae20-8ce202a02b9f-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 11, 'output_tokens': 109, 'total_tokens': 120, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}}),
